In [1]:
import pandas as pd

def plot_extended_figure4e():
    results = pd.read_csv('../data/Extended_Figure4/results.csv', index_col=0)
    print(results)

def plot_extended_figure4f():
    complexity = pd.read_csv('../data/Extended_Figure4/complexity.csv', index_col=0)
    print(complexity)

plot_extended_figure4e()

           ViT-MAE  ViT-DeIT  TokenPose (HRNet)  TokenPose (CNN)  HRNet
iRodent       26.6      28.2               40.7             39.5   40.3
AwA-OOD       38.2      38.9               55.6             53.5   53.3
TriMouse      32.4      34.2               33.4             43.5   28.1
Openfield     63.6      62.4               91.5             82.7   72.3


In [2]:
plot_extended_figure4f()

                   Params, M  MACs, G
ViT-MAE                29.80      7.5
ViT-DeIT               15.00      8.8
TokenPose (HRNet)       7.51     20.1
TokenPose (CNN)        86.00     17.6
HRNet                  86.00     17.6
